In [6]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import get_laplacian, to_dense_adj
import itertools

##### example graph

In [2]:
# COO format -> row 1 : row index, row 2 : col index
# shape (2, num_edges)
# source and target nodes of all edges
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

# node features
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

data

Data(x=[3, 1], edge_index=[2, 4])

In [3]:
# edge_index, i.e. the tensor defining the source and target nodes of all edges,
# is not a list of index tuples. 
# If you want to write your indices this way, 
# you should transpose and call contiguous on it before passing them to the data constructor

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

data

Data(x=[3, 1], edge_index=[2, 4])

Although the graph has only two edges, we need to define four index tuples to account for both directions of a edge.

Note that it is necessary that the elements in edge_index only hold indices in the range { 0, ..., num_nodes - 1}. This is needed as we want our final data representation to be as compact as possible, e.g., we want to index the source and destination node features of the first edge (0, 1) via x[0] and x[1], respectively. You can always check that your final Data objects fulfill these requirements by running validate()

In [4]:
data.validate(raise_on_error=True)

True

In [7]:
data.edge_index

tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])

In [8]:
# returns (edge_index, edge_weight)
get_laplacian(edge_index=data.edge_index)

(tensor([[0, 1, 1, 2, 0, 1, 2],
         [1, 0, 2, 1, 0, 1, 2]]),
 tensor([-1., -1., -1., -1.,  1.,  2.,  1.]))

In [10]:
to_dense_adj(data.edge_index)

tensor([[[0., 1., 0.],
         [1., 0., 1.],
         [0., 1., 0.]]])

##### complete graph

In [12]:
# vertices
nodes = [i for i in range(5)]
# edges
edges = list(itertools.permutations(range(5), 2))
edges = [list(e) for e in edges]

In [72]:
edge_index = torch.tensor(edges, dtype=torch.long)
x = torch.tensor([[0.1], [0.8], [0.8], [0.9], [0.8]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index.t().contiguous())

data

Data(x=[5, 1], edge_index=[2, 20])

In [73]:
data.edge_index

tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]])

In [74]:
to_dense_adj(data.edge_index)

tensor([[[0., 1., 1., 1., 1.],
         [1., 0., 1., 1., 1.],
         [1., 1., 0., 1., 1.],
         [1., 1., 1., 0., 1.],
         [1., 1., 1., 1., 0.]]])

In [75]:
get_laplacian(edge_index=data.edge_index)

(tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 0, 1, 2, 3,
          4],
         [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3, 0, 1, 2, 3,
          4]]),
 tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1.,  4.,  4.,  4.,  4.,  4.]))

In [76]:
L = torch.tensor([[-1.]*5]*5)
for i in range(5):
    L[i][i] = 4

L

tensor([[ 4., -1., -1., -1., -1.],
        [-1.,  4., -1., -1., -1.],
        [-1., -1.,  4., -1., -1.],
        [-1., -1., -1.,  4., -1.],
        [-1., -1., -1., -1.,  4.]])

In [77]:
L@data.x

tensor([[-2.9000],
        [ 0.6000],
        [ 0.6000],
        [ 1.1000],
        [ 0.6000]])